Import Modules


In [41]:
#Import Modules
import os, sqlite3
import pandas as pd
import numpy as np
import glob
import datetime


In [29]:
# creating file (path) name
dbfile = './DATA/BINA_DATA.db' 

# Test if the database file is available in the colab workspace
if os.path.exists(dbfile):
    # Create database (file) and Open a (SQL) connection 
    connection = sqlite3.connect(dbfile)
    # Create a data cursor to exchange information between Python and SQLite
    cursor = connection.cursor()
else:
    print("Angegebene Database wurde nicht gefunden")
    # Create database (file) and Open a (SQL) connection 
    connection = sqlite3.connect(dbfile)
    # Create a data cursor to exchange information between Python and SQLite
    cursor = connection.cursor()
    #sys.exit(0)


Angegebene Database wurde nicht gefunden


In [30]:
# Tabelle erstellen
sql = ["CREATE TABLE city (id INTEGER PRIMARY KEY, plz string, cityName string, bfsID string, lawCityName string, kantonkuerzel string)",
"CREATE TABLE smartmeter (id INTEGER PRIMARY KEY, plz REFERENCES city(plz) ON UPDATE CASCADE, timestamp string, anzMeter int,valueKwh float)",
"CREATE TABLE solarPlants (id INTEGER PRIMARY KEY, xtfID string, plz string, totalPower float, mainCategory string, subCategory string, plantCategory string,  _x int,  _y int, FOREIGN KEY (plz) REFERENCES city(plz) ON UPDATE CASCADE, FOREIGN KEY (subCategory) REFERENCES subCategory(id) ON UPDATE CASCADE, FOREIGN KEY (mainCategory) REFERENCES mainCategory(id) ON UPDATE CASCADE, FOREIGN KEY (plantCategory) REFERENCES plantCategory(id) ON UPDATE CASCADE)",
"CREATE TABLE mainCategory (id string PRIMARY KEY, de string, fr string, it string, en string)",
"CREATE TABLE plantCategory (id string PRIMARY KEY, de string, fr string, it string, en string)",
"CREATE TABLE subCategory (id string PRIMARY KEY, de string, fr string, it string, en string)",
"CREATE TABLE indicator (id string PRIMARY KEY, descr string)",
"CREATE TABLE unit (id string,  mes string)",
"CREATE TABLE demoValue (id INTEGER PRIMARY KEY, bfsID string, period string, indicator string, unit string, value float, FOREIGN KEY (bfsID) REFERENCES plz(bfsID) ON UPDATE CASCADE, FOREIGN KEY (indicator) REFERENCES indicator (id) ON UPDATE CASCADE, FOREIGN KEY (unit) REFERENCES unit(id) ON UPDATE CASCADE)"]

for code in sql:
    cursor.execute(code)
    

In [31]:
# Views erstellen
sql = [ "CREATE VIEW population AS SELECT id, bfsID, period, value FROM demoValue WHERE demoValue.indicator == 'Ind_01_01';",
        "CREATE VIEW population_density AS SELECT id, bfsID, period, value FROM demoValue WHERE demoValue.indicator == 'Ind_01_03';",
        "CREATE VIEW areaTotal AS SELECT id, bfsID, period, value FROM demoValue WHERE demoValue.indicator == 'Ind_04_01';",
        "CREATE VIEW areaSettlement AS SELECT id, bfsID, period, value FROM demoValue WHERE demoValue.indicator == 'Ind_04_02';",
        "CREATE VIEW areaAgricultural AS SELECT id, bfsID, period, value FROM demoValue WHERE demoValue.indicator == 'Ind_04_04';",
        "CREATE VIEW areaUnproductive AS SELECT id, bfsID, period, value FROM demoValue WHERE demoValue.indicator == 'Ind_04_07';",
        "CREATE VIEW keyFiguresPopulation as SELECT p.bfsID, p.period, p.value as population, pd.value as populationDensity FROM population p LEFT JOIN population_density pd ON p.bfsID = pd.bfsID AND p.period = pd.period;",
        "CREATE VIEW keyFiguresArea as SELECT a.bfsID, a.period as periodTotal, a.value as total,ase.period as periodSettlement, ase.value as settlement, aa.period as periodAgricultural, aa.value as agricultural, au.period as periodUnproductive, au.value as unproductive FROM areaTotal a LEFT JOIN areaSettlement ase ON a.bfsID = ase.bfsID LEFT JOIN areaAgricultural aa ON a.bfsID = aa.bfsID LEFT JOIN areaUnproductive au ON a.bfsID = au.bfsID;",
        "CREATE VIEW sumSmartmeter as SELECT s.plz as plz, bfsID, SUM(valueKwh) as 'kWh' FROM smartmeter as s LEFT JOIN (SELECT plz, bfsID FROM city GROUP BY plz) as c ON s.plz = c.plz GROUP BY bfsID;"
       ]

for code in sql:
    cursor.execute(code)

In [32]:
# City Daten einlesen als Dataframe und in DB sichern
city = pd.read_csv("./DATA/city_directory/AMTOVZ_CSV_LV95.csv", delimiter=";")
#city.rename(columns={"Kantonskürzel":"kantonkuerzel"})

for index, row in city.iterrows():
    ort = '"' + row["Ortschaftsname"] + '"'
    lawCityName = '"' + row["Gemeindename"] + '"'
    if isinstance(row["Kantonskürzel"], str):
        kantonkuerzel = '"' + row["Kantonskürzel"] + '"'
    else:
        kantonkuerzel = "NULL"
    sql = "INSERT INTO city VALUES(NULL,{},{},{},{},{})".format(row["PLZ"], ort, row["BFS-Nr"], lawCityName, kantonkuerzel)
    cursor.execute(sql)

connection.commit()

,Ortschaftsname,PLZ,Zusatzziffer,Gemeindename,BFS-Nr,Kantonskürzel,E,N,Sprache,Validity
0,Aeugst am Albis,8914,0,Aeugst am Albis,1,ZH,2679402.872,1235842.010,de,2008-07-01
1,Aeugstertal,8914,2,Aeugst am Albis,1,ZH,2679815.372,1237404.310,de,2008-07-01
2,Zwillikon,8909,0,Affoltern am Albis,2,ZH,2675280.133,1238108.286,de,2008-07-01
3,Affoltern am Albis,8910,0,Affoltern am Albis,2,ZH,2676852.012,1236929.718,de,2008-07-01
4,Bonstetten,8906,0,Bonstetten,3,ZH,2677412.150,1241078.278,de,2008-07-01


In [33]:
# Solar Anlagen Daten einlesen als Dataframe und in DB sichern
plantCategory = pd.read_csv("./DATA/solar_powerplants/PlantCategoryCatalogue.csv", delimiter=",")
plantCategory.head()

for index, row in plantCategory.iterrows():
    catalogueId = '"' + row["Catalogue_id"] + '"'
    de = '"' + row["de"] + '"'
    fr = '"' + row["fr"] + '"'
    it = '"' + row["it"] + '"'
    en = '"' + row["en"] + '"'
    sql = "INSERT INTO plantCategory VALUES({},{},{},{},{})".format(catalogueId, de, fr, it, en)
    cursor.execute(sql)

connection.commit()

In [34]:
# Importieren der Hauptkategorien
mainCategory = pd.read_csv("./DATA/solar_powerplants/MainCategoryCatalogue.csv", delimiter=",")
mainCategory.head()

for index, row in mainCategory.iterrows():
    catalogueId = '"' + row["Catalogue_id"] + '"'
    de = '"' + row["de"] + '"'
    fr = '"' + row["fr"] + '"'
    it = '"' + row["it"] + '"'
    en = '"' + row["en"] + '"'
    sql = "INSERT INTO mainCategory VALUES({},{},{},{},{})".format(catalogueId, de, fr, it, en)
    cursor.execute(sql)

connection.commit()

In [35]:
# Importieren der Unterkategorien
subCategory = pd.read_csv("./DATA/solar_powerplants/SubCategoryCatalogue.csv", delimiter=",")
subCategory.head()

for index, row in subCategory.iterrows():
    catalogueId = '"' + row["Catalogue_id"] + '"'
    de = '"' + row["de"] + '"'
    fr = '"' + row["fr"] + '"'
    it = '"' + row["it"] + '"'
    en = '"' + row["en"] + '"'
    sql = "INSERT INTO subCategory VALUES({},{},{},{},{})".format(catalogueId, de, fr, it, en)
    cursor.execute(sql)

connection.commit()

In [36]:
# Importieren der Solar Kraftanlagen
electricityProductionPlant = pd.read_csv("./DATA/solar_powerplants/ElectricityProductionPlant.csv", delimiter=",")
electricityProductionPlant.head()
electricityProductionPlant.replace(np.nan, "NULL", inplace=True)

for index, row in electricityProductionPlant.iterrows():
    mainCategoryId = '"' + row["MainCategory"] + '"'
    subCategoryId = '"' + row["SubCategory"] + '"'
    xCor = row["_x"]
    yCor = row["_y"]
    
    if isinstance(row["PlantCategory"], str):
        plantCategoryId = '"' + row["PlantCategory"] + '"'
    else:
        plantCategoryId = "NULL"
        
    sql = "INSERT INTO solarPlants VALUES(NULL,{},{},{},{},{},{},{},{})".format(row["xtf_id"], row["PostCode"], row["TotalPower"], mainCategoryId, subCategoryId, plantCategoryId, xCor, yCor)
    cursor.execute(sql)
    
connection.commit()

In [37]:
#Importieren Demografie Daten Indikator
demoIndicator = pd.read_csv("./DATA/key_figures_communities/Indicator_DemoData.csv", delimiter=";")

for index, row in demoIndicator.iterrows():
    indicatorId = '"' + row["INDICATORS"] + '"'
    de = '"' + row["DE"] + '"'
    
    sql = "INSERT INTO indicator VALUES({},{})".format(indicatorId, de)
    cursor.execute(sql)
    
connection.commit()

In [38]:
#Importieren Unit Messeinheit
unitMeas = pd.read_csv("./DATA/key_figures_communities/unit_mes_DemoData.csv", delimiter=";")

for index, row in unitMeas.iterrows():
    unitMeasId = '"' + row["UNIT_MES"] + '"'
    unit = '"' + row["Einheit"] + '"'
    
    sql = "INSERT INTO unit VALUES({}, {})".format(unitMeasId, unit)
    cursor.execute(sql)

connection.commit()

In [39]:
#Importieren demoValue
demoValue = pd.read_csv("./DATA/key_figures_communities/ts-x-21.03.01.csv", delimiter=";", dtype={'PERIOD_REF': str, 'PERIOD_COMP': str, 'REGION': str})
demoValue.replace("CH", 0, inplace=True)
demoValue.replace(np.nan, "NULL", inplace=True)

for index, row in demoValue.iterrows():
    period = row["PERIOD_REF"]
    if isinstance(period, str):
        period = '"' + period + '"'
    else:
        period = str(period)
        period = '"' + period + '"'
        
    indicator = '"' + row["INDICATORS"] + '"'
    unit = '"' + row["UNIT_MES"] + '"'
    
    sql = "INSERT INTO demoValue VALUES(NULL,{},{},{},{},{})".format(row["CODE_REGION"], period, indicator, unit, row["VALUE"])
    cursor.execute(sql)
    
connection.commit()

In [40]:
# Importieren SmartMeter DATA
path = './DATA/smartmeter'
csv_files = glob.glob(path + '/*.csv.gz')
df_list = (pd.read_csv(file) for file in csv_files)

smartmeter_df = pd.concat(df_list, ignore_index=True)

for index, row in smartmeter_df.iterrows():
    datetime = '"' + row["timestamp"] + '"'
    sql = "INSERT INTO smartmeter VALUES(NULL,{},{},{},{})".format(row["area_code"], datetime, row["num_meter"], row["value_kwh"])
    cursor.execute(sql)
    
connection.commit()


In [54]:
#Test Date Time

input = '2020012400' + '00'
output = '%Y%m%d%H%M'

date = datetime.datetime.strptime(input, output).strftime('%Y-%m-%d %H:%M')

print(date)

2020-01-24 00:00


In [ ]:
#Import MeteoData
